<a href="https://colab.research.google.com/github/adarsh-meher/MachineHack---Predict-Delivery-Time/blob/master/MH_Predict_Delivery_Time.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Installing libraries**

In [1]:
##### Installing required libraries
!pip install hyperopt
!pip install h2o
!pip install category_encoders
!pip install rgf-python
!pip install imblearn

     |████████████████████████████████| 123.4MB 94kB/s 
  Created wheel for h2o: filename=h2o-3.28.0.1-py2.py3-none-any.whl size=123410554 sha256=5f7e82f663ce7ea181df50387ecd5515cc1b07469ac2c3993c89a15ec810b304
  Stored in directory: /root/.cache/pip/wheels/ff/82/f4/3fe9c895b93c53b25ae44dc44aa54dad571c711f48bc485fe9
Successfully built h2o
     |████████████████████████████████| 102kB 4.0MB/s 
     |████████████████████████████████| 768kB 4.9MB/s 


**Clone data from github repo to local drive**

In [2]:
! git clone https://github.com/adarsh-meher/MachineHack---Predict-Delivery-Time.git
import os
os.chdir('/content/MachineHack---Predict-Delivery-Time')

Cloning into 'MachineHack---Predict-Delivery-Time'...
remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 14 (delta 3), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (14/14), done.


In [0]:
import pandas as pd
import numpy as np
import h2o
import hyperopt as hopt
import scipy as sc
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sb
import time

In [0]:
from sklearn.model_selection import train_test_split,GridSearchCV,cross_val_score,ParameterGrid
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier,ExtraTreesClassifier
from sklearn.multiclass import OutputCodeClassifier,OneVsRestClassifier
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import RFECV
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import accuracy_score

In [0]:
train_data = pd.ExcelFile('Data_Train.xlsx').parse('Sheet1')
test_data = pd.ExcelFile('Data_Test.xlsx').parse('Sheet1')

In [7]:
train_data.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes


In [0]:
#### Concatenating train and test data.
train_data['Type'] = ['train']*train_data.shape[0]
test_data['Type'] = ['test']*test_data.shape[0]
total_data = pd.concat([train_data.drop(['Delivery_Time'],axis = 1),test_data],axis = 0,ignore_index = True)

In [22]:
#### Creating list of all uniques cuisines across train + test data
total_data['Cuisines_List'] = total_data['Cuisines'].map(lambda x : [ a.strip() for a in x.split(',') ] )
list_cuisines = []
for x in total_data['Cuisines_List']:
  for t in x:
    list_cuisines.append(t)

uniq_cuisines = list(dict.fromkeys(list_cuisines))
print('Number of unique cuisines : %s' % len(uniq_cuisines))

Number of unique cuisines : 101


In [23]:
#### Creating columns of cuisines with 0/1 entry indicating the restaurant served the cuisine(1) or not (0)
start = time.time()
for c in uniq_cuisines:
  total_data[c] = total_data['Cuisines_List'].map(lambda x : 1 if c in x else 0)
end = time.time()
print(end-start)

0.5163254737854004


In [0]:
def conv_str_to_float(x):
  if len(x) ==1:
    try:
      return int(x[0])
    except Exception as e:
      return 0.0
  else:
    t = ''.join(x[1].split(',')) if ',' in x[1] else x[1]
    return float(t)

total_data['Average_Cost_New'] = total_data['Average_Cost'].apply(lambda x : conv_str_to_float(x.split('₹')))
total_data['Minimum_Order_New'] = total_data['Minimum_Order'].apply(lambda x : conv_str_to_float(x.split('₹')))

In [0]:
def conv_str_to_float_2(x):
  try:
    return float(x)
  except Exception as e:
    return -1.0

total_data['Rating_New'] = total_data['Rating'].apply(lambda x : conv_str_to_float_2(x))
total_data['Votes_New'] = total_data['Votes'].apply(lambda x : conv_str_to_float_2(x))
total_data['Reviews_New'] = total_data['Reviews'].apply(lambda x : conv_str_to_float_2(x))

In [0]:
#total_data['Location_Str'] = total_data['Location']
#total_data = pd.get_dummies(total_data,columns = ['Location'])

In [27]:
#### Creating train/test split. Features creation and model training will be done on train and performance evaluation will be done on test
train_new,test_new = total_data[total_data['Type'] == 'train'],total_data[total_data['Type'] == 'test']
train_new.index = range(train_new.shape[0])
test_new.index = range(test_new.shape[0])

train_new1,test_new1,target_train,target_test = train_test_split(train_new,train_data['Delivery_Time'],test_size = 0.25,stratify = train_data['Delivery_Time'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [42]:
from sklearn.preprocessing import LabelEncoder
le_fit = LabelEncoder().fit(target_train)
target_enc = le_fit.transform(target_train)
test_enc = le_fit.transform(target_test)
train_new1['Delivery_Time_Enc'] = target_train

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [36]:
print(pd.Series(target_enc).value_counts())
print(pd.Series(test_enc).value_counts())

3    5554
4    1999
5     692
1      46
2      15
6      11
0       3
dtype: int64
3    1852
4     666
5     231
1      16
2       5
6       3
0       1
dtype: int64


In [39]:
print('Unique restaurants in train : %s' % (len(set(train_new1['Restaurant']))))
print('Unique restaurants in test : %s' % (len(set(test_new1['Restaurant']))))
print('Restaurants in both train/test : %s' % (len(set(train_new1['Restaurant']).intersection(set(test_new1['Restaurant'])))))

Unique restaurants in train : 6038
Unique restaurants in test : 2428
Restaurants in both train/test : 986


***Analyzing variables for feature engineering***
---




In [38]:
train_data.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,Type
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes,train
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes,train
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",₹150,₹50,3.6,99,30,65 minutes,train
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",₹250,₹99,3.7,176,95,30 minutes,train
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",₹200,₹99,3.2,521,235,65 minutes,train


In [43]:
####  Hypothesis 1 : Variation in average cost vs delivery time
train_new1.groupby(['Delivery_Time_Enc'])['Average_Cost_New'].agg({'Mean' :  np.mean,'Median' : np.median,'Min' : np.min,'Max' : np.max,'STDEV' : np.std})

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: using a dict on a Series for aggregation
is deprecated and will be removed in a future version. Use                 named aggregation instead.

    >>> grouper.agg(name_1=func_1, name_2=func_2)

  """Entry point for launching an IPython kernel.


,Mean,Median,Min,Max,STDEV
Delivery_Time_Enc,,,,,
10 minutes,133.333333,150.0,100.0,150.0,28.867513
120 minutes,175.000000,200.0,100.0,300.0,49.159604
20 minutes,183.333333,200.0,100.0,250.0,40.824829
30 minutes,183.678430,150.0,0.0,2050.0,111.614343
45 minutes,235.017509,200.0,50.0,1200.0,148.716185
65 minutes,258.526012,200.0,50.0,1200.0,159.207046
80 minutes,240.909091,200.0,100.0,400.0,111.396099


In [50]:
#### Summary stats of delivery time vs Location
loc_time_grp = train_new1.groupby(['Location'])['Delivery_Time_Enc']
df1 = pd.DataFrame()
for ind,grps in loc_time_grp:
  val = []
  for i in grps.values:
    t,_= i.split(' ')
    val.append(float(t))
  
  df2 = pd.DataFrame({'Location' : ind,'Mean' : np.mean(val),'Median' : np.median(val),'Min' : np.min(val),'Max' : np.max(val),'STDEV' : np.std(val)},index = [0])
  df1 = pd.concat([df1,df2],axis = 0)

df1.sort_values(ascending = False,by = ['Mean'])

,Location,Mean,Median,Min,Max,STDEV
0,"BTM Layout 1, Electronic City",41.666667,45.0,30.0,65.0,12.472191
0,"Sector 63A,Gurgaon",41.465969,30.0,30.0,120.0,19.183535
0,"Delhi High Court, India Gate",40.709571,30.0,20.0,120.0,13.921330
0,"Moulali, Kolkata",39.356436,30.0,30.0,120.0,14.240915
0,"Chandni Chowk, Kolkata",39.183673,30.0,30.0,120.0,14.422451
0,"Sector 1, Noida",38.829787,30.0,10.0,120.0,15.784883
0,"Sector 14, Noida",38.582888,30.0,10.0,120.0,14.283131
0,Mumbai Central,38.460366,30.0,30.0,120.0,12.096851
0,"Raja Bazar, Kolkata",38.409091,30.0,30.0,120.0,14.549005
0,"Sandhurst Road, Mumbai CST Area",37.974453,30.0,30.0,120.0,12.808360


,Location,Mean,Median,Min,Max
0,"BTM Layout 1, Electronic City",40.909091,45.0,30.0,65.0
0,"Babarpur, New Delhi, Delhi",31.773050,30.0,30.0,65.0
0,"Chandni Chowk, Kolkata",39.285714,30.0,30.0,120.0
0,"Chatta Bazaar, Malakpet, Hyderabad",34.259259,30.0,30.0,65.0
0,"D-Block, Sector 63, Noida",37.450372,30.0,10.0,120.0
0,"Delhi Administration Flats, Timarpur",36.027542,30.0,30.0,120.0
0,Delhi Cantt.,36.579755,30.0,30.0,65.0
0,"Delhi High Court, India Gate",41.277372,30.0,20.0,120.0
0,Delhi University-GTB Nagar,36.440367,30.0,20.0,120.0
0,"Dockyard Road, Mumbai CST Area",37.216495,30.0,30.0,120.0


In [0]:
train_new1,val_new1,train_target,val_target = train_test_split(train_new,target_enc,test_size = 0.2,stratify = target_enc)

In [0]:
print(train_data['Delivery_Time_Enc'].value_counts())
print(pd.Series(val_target).value_counts())

3    7406
4    2665
5     923
1      62
2      20
6      14
0       4
Name: Delivery_Time_Enc, dtype: int64
3    1481
4     533
5     185
1      12
2       4
6       3
0       1
dtype: int64


In [0]:
del_time_dict = dict(train_data['Delivery_Time_Enc'].value_counts())
modal_class = train_data['Delivery_Time_Enc'].value_counts().argmax()
modal_class_value = train_data['Delivery_Time_Enc'].value_counts().max()
del del_time_dict[modal_class]
for x in del_time_dict.keys():
  if x != modal_class:
    del_time_dict[x] = modal_class_value

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  


In [0]:
numeric_cols = total_data.select_dtypes(include = [int,float]).columns.tolist()
from imblearn.over_sampling import RandomOverSampler
train_resamp,target_resamp = RandomOverSampler(sampling_strategy=del_time_dict,random_state = 1993).fit_resample(train_new[numeric_cols],target_enc)
train_resamp,target_resamp = pd.DataFrame(train_resamp,index = range(train_resamp.shape[0]),columns = numeric_cols),pd.Series(target_resamp,index = range(len(target_resamp)))

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [0]:
print(train_data['Delivery_Time_Enc'].value_counts())
print(train_data['Delivery_Time_Enc'].value_counts()/train_data.shape[0])
print(pd.Series(target_resamp).value_counts())

3    7406
4    2665
5     923
1      62
2      20
6      14
0       4
Name: Delivery_Time_Enc, dtype: int64
3    0.667568
4    0.240220
5    0.083198
1    0.005589
2    0.001803
6    0.001262
0    0.000361
Name: Delivery_Time_Enc, dtype: float64
3    7406
6    2665
5    2665
4    2665
2    2665
1    2665
0    2665
dtype: int64


In [0]:
train_resamp.shape

(23396, 106)

In [0]:
xtrain,xval,ytrain,yval = train_test_split(train_resamp,target_resamp,test_size = 0.25,stratify = target_resamp)

In [0]:
clf = RandomForestClassifier(random_state = 1993)
rfe = RFECV(estimator = clf,cv = 5,scoring = 'accuracy',verbose = 2)
rfe_fit = rfe.fit(xtrain,ytrain)

Fitting estimator with 106 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 105 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 104 features.


...

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 35 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Fitting estimator with 34 features.


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [0]:
#feat_selected = train_resamp[numeric_cols].columns[rfe_fit.support_]
clf = RandomForestClassifier(random_state = 1993)
occ = OneVsRestClassifier(clf)
occ_fit = occ.fit(xtrain,ytrain)
cross_val_score(occ,xtrain,ytrain,cv = 5,scoring = 'accuracy',verbose = 2)

/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[CV]  ................................................................


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[CV] ................................................. , total=   0.8s


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.8s remaining:    0.0s


...

[CV] ................................................. , total=   0.8s
[CV]  ................................................................


/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

[CV] ................................................. , total=   0.8s


[Parallel(n_jobs=1)]: Done   5 out of   5 | elapsed:    3.9s finished


array([0.89547138, 0.89689547, 0.89888921, 0.897151  , 0.90896119])

In [0]:
occ_preds = occ_fit.predict(xval)
print('Accuracy Score : %s' % accuracy_score(yval,occ_preds) )

NameError: ignored

In [0]:
#### Fitting model on entire train
model_fit = occ.fit(train_resamp,target_resamp)
model_preds = model_fit.predict(test_new[numeric_cols])
model_preds_1 = le_fit.inverse_transform(model_preds)
output_df = pd.DataFrame({'Delivery_Time' : model_preds_1},index = range(len(model_preds)))

In [0]:
output_df.to_excel('Submission_LGBM_1.xlsx',index = False)